[Corpora and Vector Spaces](https://radimrehurek.com/gensim/tut1.html)

[Tutorial 1: Corpora and Vector Spaces](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Corpora_and_Vector_Spaces.ipynb)


In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from gensim import corpora

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

2018-05-10 16:38:59,928 : INFO : 'pattern' package not found; tag filters are not available for English


In [4]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [8]:
import os

dictionary = corpora.Dictionary(texts)
# file_dir = os.path.dirname(os.path.abspath(__file__))
# file_dir = os.getcwd()
# fn = os.path.join(file_dir, 'data/deerwester.dict')
# dictionary.save(fn)  # store the dictionary, for future reference
print(dictionary)

2018-05-10 16:51:12,584 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-10 16:51:12,586 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [9]:
dictionary.token2id

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}

In [10]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(0, 1), (1, 1)]


以上是简单的频数，如果用one hot encoding来得到句向量的话，坐标的具体的值就是频数的值。如果简单这样做的话，频数包含了太多的噪音，可能还不如用0, 1来表示句子当中有没有出现这个词。

In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())


[Topics and Transformations](https://radimrehurek.com/gensim/tut2.html)

In [13]:
from gensim import models, similarities

tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2018-05-10 17:19:57,636 : INFO : collecting document frequencies
2018-05-10 17:19:57,639 : INFO : PROGRESS: processing document #0
2018-05-10 17:19:57,641 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [14]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])

[(0, 0.70710678118654757), (1, 0.70710678118654757)]


In [15]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)


[(0, 0.57735026918962573), (1, 0.57735026918962573), (2, 0.57735026918962573)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.32448702061385548), (6, 0.44424552527467476), (7, 0.32448702061385548)]
[(2, 0.5710059809418182), (5, 0.41707573620227772), (7, 0.41707573620227772), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.71848116070837686), (8, 0.49182558987264147)]
[(3, 0.62825804686700459), (6, 0.62825804686700459), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.70710678118654746), (10, 0.70710678118654746)]
[(9, 0.50804290089167492), (10, 0.50804290089167492), (11, 0.69554641952003704)]
[(4, 0.62825804686700459), (10, 0.45889394536615247), (11, 0.62825804686700459)]


以上是各个句子中词汇的tf-idf的值，tf是针对这个句子算的，idf是针对所有句子的集合来算的，如果用one hot encoding来得到句向量的话，坐标的具体的值就是tf-idf的值

Term Frequency * Inverse Document Frequency, Tf-Idf expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality, except that features which were rare in the training corpus will have their value increased. It therefore converts integer-valued vectors into real-valued ones, while leaving the number of dimensions intact. It can also optionally normalize the resulting vectors to (Euclidean) unit length.

    model = models.TfidfModel(corpus, normalize=True)

In [16]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-05-10 17:27:38,029 : INFO : using serial LSI version on this node
2018-05-10 17:27:38,031 : INFO : updating model with new documents
2018-05-10 17:27:38,033 : INFO : preparing a new chunk of documents
2018-05-10 17:27:38,035 : INFO : using 100 extra samples and 2 power iterations
2018-05-10 17:27:38,036 : INFO : 1st phase: constructing (12, 102) action matrix
2018-05-10 17:27:38,041 : INFO : orthonormalizing (12, 102) action matrix
2018-05-10 17:27:38,069 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-05-10 17:27:38,092 : INFO : computing the final decomposition
2018-05-10 17:27:38,095 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2018-05-10 17:27:38,101 : INFO : processed documents up to #9
2018-05-10 17:27:38,105 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-05-10 17:27:38,107 : INFO : topic #

Here we transformed our Tf-Idf corpus via Latent Semantic Indexing into a latent 2-D space (2-D because we set num_topics=2). Now you’re probably wondering: what do these two latent dimensions stand for? Let’s inspect with models.LsiModel.print_topics():

In [17]:
lsi.print_topics(2)

2018-05-10 18:07:21,973 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-05-10 18:07:21,977 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

lsi的两个维度是之前的词汇维度线性组合的结果，可以认为是对词汇维度降维的结果，于是，句向量也就降到了这两个维度中

In [18]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

[(0, 0.066007833960905843), (1, -0.52007033063618557)]
[(0, 0.19667592859142821), (1, -0.76095631677000375)]
[(0, 0.089926399724467476), (1, -0.72418606267525132)]
[(0, 0.075858476521784332), (1, -0.63205515860034378)]
[(0, 0.10150299184980362), (1, -0.57373084830029475)]
[(0, 0.70321089393782998), (1, 0.16115180214026084)]
[(0, 0.87747876731198193), (1, 0.16758906864659784)]
[(0, 0.90986246868185683), (1, 0.14086553628719414)]
[(0, 0.61658253505692839), (1, -0.053929075663890824)]


以上给出了各个句子的句向量，这个lsi模型是二维的，因此只有两个坐标

LSI模型与LSA模型是一样的，Latent Semantic Indexing和Latent Semantic Analysis说的是一回事。

Latent Semantic Indexing, LSI (or sometimes LSA) transforms documents from either bag-of-words or (preferrably) TfIdf-weighted space into a latent space of a lower dimensionality. For the toy corpus above we used only 2 latent dimensions, but on real corpora, target dimensionality of 200–500 is recommended as a “golden standard”

    model = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=300)
    
LSI training is unique in that we can continue “training” at any point, simply by providing more training documents. This is done by incremental updates to the underlying model, in a process called online training. Because of this feature, the input document stream may even be infinite – just keep feeding LSI new documents as they arrive, while using the computed transformation model as read-only in the meanwhile!

    model.add_documents(another_tfidf_corpus) # now LSI has been trained on tfidf_corpus + another_tfidf_corpus
    lsi_vec = model[tfidf_vec] # convert some new document into the LSI space, without affecting the model
    ...
    model.add_documents(more_documents) # tfidf_corpus + another_tfidf_corpus + more_documents
    lsi_vec = model[tfidf_vec]

In [19]:
model_lda = models.LdaModel(corpus, id2word=dictionary, num_topics=100)
corpus_lda = model_lda[corpus]
for doc in corpus_lda:
    print(doc)

2018-05-10 18:33:17,018 : INFO : using symmetric alpha at 0.01
2018-05-10 18:33:17,020 : INFO : using symmetric eta at 0.01
2018-05-10 18:33:17,022 : INFO : using serial LDA version on this node
2018-05-10 18:33:17,027 : INFO : running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-10 18:33:17,029 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-05-10 18:33:17,038 : INFO : -124.837 per-word bound, 37980155611459346870173441551641870336.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2018-05-10 18:33:17,039 : INFO : PROGRESS: pass 0, at document #9/9
2018-05-10 18:33:17,045 : INFO : topic #62 (0.010): 0.083*"user" + 0.083*"system" + 0.083*"graph" + 0.083*"trees" + 0.083*"eps" + 0.

[(91, 0.75250006)]
[(23, 0.85857147)]
[(13, 0.80200005)]
[(67, 0.80200005)]
[(28, 0.7525)]
[(94, 0.505)]
[(64, 0.67000002)]
[(70, 0.7525)]
[(87, 0.75250006)]


Latent Dirichlet Allocation, LDA is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

输出的结果似乎是直接给出所属的主题分类（在各个类中概率最大的主题类别），并且把该类的概率给出了。应该还有其他接口可以把所有分类的概率都给出来，也就是给一个高维向量。
